In [1]:
#CNN - Inception v4
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
#import the dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
#Transform to tensor dataset
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test))
#each element contains two subelement, including x (32, 32, 3) and y (1 value)

170508288/170498071 [==============================] - 4s 0us/step


In [3]:
#preprocess the data
def process_image(image,label):
    image=tf.image.per_image_standardization(image)
    image=tf.image.resize(image,(299,299))
    
    return image,label
train_ds_size=tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size=tf.data.experimental.cardinality(test_ds).numpy()
print('Train size:',train_ds_size)
print('Test size:',test_ds_size)

Train size: 50000
Test size: 10000


In [4]:
#Data transformation
train_ds=(train_ds
          .map(process_image)
          .shuffle(buffer_size = 45000)
          .batch(batch_size=32,drop_remainder=True)
         )
test_ds=(test_ds
          .map(process_image)
          .shuffle(buffer_size=10000)
          .batch(batch_size=32,drop_remainder=True)
         )

In [5]:
#normal conv with bn and activation
def conv2d(input_layer, filters, kernel_tuple, strides=1, padding="same", use_bias=False):
    #each conv2d would have bn and activation
    conv = layers.Conv2D(filters=filters, kernel_size=kernel_tuple, \
                        strides=strides, padding=padding, use_bias=use_bias)(input_layer)
    bn = layers.BatchNormalization(momentum=0.9997, scale=False)(conv)
    activation = layers.Activation('relu')(bn)
    return activation

#stem part
def stem(input_layer, n=7):
    x = conv2d(input_layer, 32, 3, strides=2, padding="valid")
    x = conv2d(x, 32, 3, padding="valid")
    x = conv2d(x, 64, 3)
    #split
    conv1 = conv2d(x, 96, 3, strides=2, padding="valid")
    maxpool = layers.MaxPooling2D(pool_size=3, strides=2, padding='valid')(x)
    #filter concat
    x = layers.concatenate([conv1, maxpool], axis=-1)
    #split
    #line 1
    conv1_1 = conv2d(x, 64, 1)
    conv1_2 = conv2d(conv1_1, 96, 3, padding='valid')
    #line 2
    conv2_1 = conv2d(x, 64, 1)
    conv2_2 = conv2d(conv2_1, 64, (n, 1))
    conv2_3 = conv2d(conv2_2, 64, (1, n))
    conv2_4 = conv2d(conv2_3, 96, 3, padding='valid')
    #filter concat
    x = layers.concatenate([conv1_2, conv2_4], axis=-1)
    
    return x

def inceptionA(input_layer):
    #line 1
    avg = layers.AveragePooling2D(pool_size = 3,strides = 1,padding = 'same')(input_layer)
    conv1_1 = conv2d(avg, 96, 1)
    #line 2
    conv2_1 = conv2d(input_layer, 96, 1)
    #line 3
    conv3_1 = conv2d(input_layer, 64, 1)
    conv3_2 = conv2d(conv3_1, 96, 3)
    #line 4
    conv4_1 = conv2d(input_layer, 64, 1)
    conv4_2 = conv2d(conv4_1, 96, 3)
    conv4_3 = conv2d(conv4_2, 96, 3)
    #filter concat
    x = layers.concatenate([conv1_1, conv2_1, conv3_2, conv4_3], axis=-1)
    
    return x

def inceptionB(input_layer, n=7):
    #line 1
    avg = layers.AveragePooling2D(pool_size = 3,strides = 1,padding = 'same')(input_layer)
    conv1_1 = conv2d(avg, 128, 1)
    #line 2
    conv2_1 = conv2d(input_layer, 384, 1)
    #line 3
    conv3_1 = conv2d(input_layer, 192, 1)
    conv3_2 = conv2d(conv3_1, 224, (1, n))
    conv3_3 = conv2d(conv3_2, 256, (1, n))
    #line 4
    conv4_1 = conv2d(input_layer, 192, 1)
    conv4_2 = conv2d(conv4_1, 192, (1, n))
    conv4_3 = conv2d(conv4_2, 224, (n, 1))
    conv4_4 = conv2d(conv4_3, 224, (1, n))
    conv4_5 = conv2d(conv4_4, 256, (n, 1))
    #filter concat
    x = layers.concatenate([conv1_1, conv2_1, conv3_3, conv4_5], axis=-1)
    
    return x

def inceptionC(input_layer, n=3):
    #line 1
    avg = layers.AveragePooling2D(pool_size = 3,strides = 1,padding = 'same')(input_layer)
    conv1_1 = conv2d(avg, 256, 1)
    #line 2
    conv2_1 = conv2d(input_layer, 256, 1)
    #line 3
    conv3 = conv2d(input_layer, 384, 1)
    #line 3-1
    conv3_1 = conv2d(conv3, 256, (1, n))
    #line 3-2
    conv3_2 = conv2d(conv3, 256, (n, 1))
    #line 4
    conv4_1 = conv2d(input_layer, 384, 1)
    conv4_2 = conv2d(conv4_1, 448, (1, n))
    conv4_3 = conv2d(conv4_2, 512, (n, 1))
    #line 4a
    conv4a = conv2d(conv4_3, 256, (n, 1))
    #line 4b
    conv4b = conv2d(conv4_3, 256, (1, n))
    #filter concat
    x = layers.concatenate([conv1_1, conv2_1, conv3_1, conv3_2, conv4a, conv4b], axis=-1)
    
    return x

def reductionA(input_layer, k=192, l=224, m=256, n=384):
    #maxpool
    maxpool = layers.MaxPooling2D(pool_size=3, strides=2, padding='valid')(input_layer)
    #line 1
    conv1_1 = conv2d(input_layer, n, 3, strides=2, padding='valid')
    #line 2
    conv2_1 = conv2d(input_layer, k, 1)
    conv2_2 = conv2d(conv2_1, l, 3)
    conv2_3 = conv2d(conv2_2, m, 3, strides=2, padding='valid')
    #filter concat
    x = layers.concatenate([conv1_1, conv2_3, maxpool], axis=-1)
    
    return x

def reductionB(input_layer):
    #maxpool
    maxpool = layers.MaxPooling2D(pool_size=3, strides=2, padding='valid')(input_layer)
    #line 1
    conv1_1 = conv2d(input_layer, 192, 1)
    conv1_2 = conv2d(conv1_1, 192, 3, strides=2, padding='valid')
    #line 2
    conv2_1 = conv2d(input_layer, 256, 1)
    conv2_2 = conv2d(conv2_1, 256, (1, 7))
    conv2_3 = conv2d(conv2_2, 320, (7, 1))
    conv2_4 = conv2d(conv2_3, 320, 3, strides=2, padding='valid')
    #filter concat
    x = layers.concatenate([conv1_2, conv2_4, maxpool], axis=-1)
    
    return x


In [6]:
def inceptionV4(input_shape, classes):
    _input = _input = keras.layers.Input(shape=input_shape)
    #stem
    x = stem(_input)
    #4 x Inception-A
    for _ in range(4):
        x = inceptionA(x)
    #Reduction-A
    x = reductionA(x)
    #7 x Inception-B
    for _ in range(7):
        x = inceptionB(x)
    #reduction-B
    x = reductionB(x)
    #3 x Inception-C
    for _ in range(3):
        x = inceptionC(x)
    #avgpooling
    x = layers.AveragePooling2D(8)(x)
    #dropout
    x = layers.Dropout(0.2)(x)
    x = layers.Flatten()(x)
    #output
    output = layers.Dense(classes, activation='softmax')(x)
    return keras.Model(_input, output, name="Inception-v4")


In [7]:
Inceptionv4 = inceptionV4((299,299,3), classes=10)
Inceptionv4.summary()

Model: "Inception-v4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [8]:
Inceptionv4.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/MLh5/inceptionv4.h5'
model_checkpoint_callback = [tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True)]
#5 epochs (each epoch need around X mins, it is time-consuming.)
history = Inceptionv4.fit(
    train_ds,
    epochs=10,
    validation_data = test_ds,
    validation_freq=1,
    callbacks=model_checkpoint_callback
)

#RAM exhaustion...............................cant train the model

Epoch 1/10


In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 2.3) # set the vertical range to [0-1]
plt.show()